In [ ]:
#Importing Required Packages
import os
import pandas as pd
from pandas import DataFrame,Series
import numpy as np
import wbdata as wbd
import datetime
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from statsmodels.regression.linear_model import OLS
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LassoLarsIC as lasso_ic
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.preprocessing import scale
from patsy import dmatrices

In [ ]:
#setting the current working directory
os.chdir(r"C:\Users\conor\OneDrive\Documents\Ec & Fi Undergrad\APRM Individual Project")

In [ ]:
#Reading in the dependent variable data
#Obtained from worldometer.info/coronavirus using waybackmachine
dep_vars = pd.read_excel("Dep Vars.xlsx",sheet_name="13th April")
dep_vars.head()

In [ ]:
#Dropping unrequired columns
drop_cols = dep_vars.columns
dep_vars.drop(columns = drop_cols.drop\
              (labels=["Country","Total Cases","Total Deaths"]),inplace=True)
dep_vars.head()

In [ ]:
dep_vars.shape

In [ ]:
dep_vars.info()

In [ ]:
#dep_vars.set_index("Country",inplace=True)
dep_vars.sort_index(inplace=True)
dep_vars

In [ ]:
#Obtaining data from the World Bank
#creating a list of countries which we require data for
countries_list = dep_vars.Country.copy()
country_id_name = []
country_no_id = []
country_multiple_ids = []

#searching for each countries ID in the world bank database
for i in countries_list:
    x = wbd.search_countries(i,display=False)
    if len(x) == 0:
        country_no_id.append(i)
    elif len(x) > 1:
        country_multiple_ids.append(i)   
    else:
        for j in x:
             country_name = j["name"]
             country_id = j["id"]
             country_id_name.append([i,country_name,country_id])

In [ ]:
#printing which countries the search was unable to find IDs for
country_no_id_ser = Series(country_no_id)

In [ ]:
#creating a map to change the country names and perform another search
keys_no_id = country_no_id.copy()
values_no_id = ["United States","Korea, Rep.","Czech Republic","United Arab Emirates",\
          "Slovak Republic","Diamond Princess Cruise","Cote d'Ivoire","Kyrgyz Republic",\
          "Réunion","Palestine","Congo, Dem. Rep.","Mayotte","Faroe Islands",\
          "Martinique","Guadeloupe","French Guiana","St. Martin","Lao PDR",\
          "St. Lucia","Curacao","St. Kitts and Nevis","St. Vincent and the Grenadines",\
          "Montserrat","MS Zaandam Cruise","Vatican City","St. Barth","Western Sahara",\
          "Falkland Islands","Anguilla","Caribbean Netherlands","Saint Pierre Miquelon"]

country_map_no_id = dict(zip(keys_no_id,values_no_id))
country_no_id_ser = country_no_id_ser.map(country_map_no_id)
#search again using new country names
country_no_id = []
for i in country_no_id_ser:
    x = wbd.search_countries(i,display=False)
    if len(x) == 0:
        country_no_id.append(i)
    elif len(x) > 1:
        country_multiple_ids.append(i)   
    else:
        for j in x:
             country_name = j["name"]
             country_id = j["id"]
             country_id_name.append([i,country_name,country_id])


In [ ]:
#Printing which countries where multiple IDs were found
country_multi_ser = Series(country_multiple_ids)


In [ ]:
#creating a map to cahnge the country names and perform another search           
keys_multi = country_multiple_ids.copy()
values_multi = ["China","South Africa","Oman","Niger","Nigeria","Guinea","Mali"\
                ,"Congo, Rep.","Sudan","Dominica","Central African Republic"]
values_multi_kf = ["China","Oman","Niger","Nigeria","Guinea","Mali","Sudan","Dominica"]
values_multi_kl = ["South Africa","Congo, Rep."]

country_map_multi = dict(zip(keys_multi,values_multi))
country_multi_ser = country_multi_ser.map(country_map_multi)

#search again using new country names
for i in country_multi_ser:
    x = wbd.search_countries(i,display=False)
    if i in values_multi_kf:
        country_name = x[0]["name"]
        country_id = x[0]["id"]
        country_id_name.append([i,country_name,country_id])
    elif i in values_multi_kl:
        country_name = x[-1]["name"]
        country_id = x[-1]["id"]
        country_id_name.append([i,country_name,country_id])
    else:
         country_name = x[0]["name"]
         country_id = x[0]["id"]
         country_id_name.append([i,country_name,country_id])


In [ ]:
country_df = DataFrame(country_id_name,\
                       columns=["Dep Vars Names","WBD Name","WBD ID"])
country_df.set_index("Dep Vars Names",inplace=True)
country_df.sort_index(inplace=True)
country_df.head()

In [ ]:
#Checking for duplicates
country_df.index.duplicated().sum()

In [ ]:
#Checking shape of the country dataframe
country_df.shape

In [ ]:
#Number of countries that do not have an ID
len(country_no_id)

In [ ]:
#Creating a list of country IDs
country_id = country_df["WBD ID"].tolist()

In [ ]:
#Creating a list of country names
country_name = country_df["WBD Name"].tolist()

In [ ]:
#Printing list of data topics from the World Bank
wbd.get_topic()

In [ ]:
#Agriculture and Rural Development
wbd.search_indicators("",topic=1)

In [ ]:
#Aid Effectiveness
wbd.search_indicators("",topic=2)

In [ ]:
#Economy and Growth
wbd.search_indicators("",topic=3)

In [ ]:
#Education
wbd.search_indicators("",topic=4)

In [ ]:
#Energy and Mining
wbd.search_indicators("",topic=5)

In [ ]:
#Environment
wbd.search_indicators("",topic=6)

In [ ]:
#Financial Services
wbd.search_indicators("",topic=7)

In [ ]:
#Health
wbd.search_indicators("",topic=8)

In [ ]:
#Infrastructure
wbd.search_indicators("",topic=9)

In [ ]:
#Social Protection and Labour
wbd.search_indicators("",topic=10)

per_si_allsi.avt_q5_tot       	Average per capita transfer held by 5th quintile (richest) - All Social Insurance
per_si_allsi.avt_q5_urb       	Average per capita transfer held by 5th quintile (richest) - All Social Insurance -urban
per_si_allsi.ben_ep_preT_tot  	Benefits incidence in extreme poor (<$1.90 PPP a day) (%) - All Social Insurance  (preT)
per_si_allsi.ben_ep_tot       	Benefits incidence in extreme poor (<$1.90 PPP a day) (%) - All Social Insurance
per_si_allsi.ben_q1_preT_tot  	Benefits incidence in 1st quintile (poorest) (%) - All Social Insurance (preT)
per_si_allsi.ben_q1_rur       	Benefits incidence in 1st quintile (poorest) (%) - All Social Insurance -rural
per_si_allsi.ben_q1_tot       	Benefit incidence of social insurance programs to poorest quintile (% of total social insurance benefits)
per_si_allsi.ben_q1_urb       	Benefits incidence in 1st quintile (poorest) (%) - All Social Insurance -urban
per_si_allsi.ben_q2_preT_tot  	Benefits incidence in 2nd quintile (%

In [ ]:
#Poverty
wbd.search_indicators("",topic=11)

In [ ]:
#Private Sector
wbd.search_indicators("",topic=12)

In [ ]:
#Public Sector
wbd.search_indicators("",topic=13)

In [ ]:
#Science and Technology
wbd.search_indicators("",topic=14)

In [ ]:
#Social Development
wbd.search_indicators("",topic=15)

In [ ]:
#Urban Development
wbd.search_indicators("",topic=16)

In [ ]:
#Gender
wbd.search_indicators("",topic=17)

In [ ]:
#Millenium Development Goals
wbd.search_indicators("",topic=18)

In [ ]:
#Climate Change
wbd.search_indicators("",topic=19)

In [ ]:
#Importing list of Indicator IDs
indicators_df = pd.read_excel("World Bank Indicators.xlsx",sheet_name="Sheet1")
indicators_df.head()

In [ ]:
indicators_df.shape

In [ ]:
#Dropping any duplicates and NaN values
indicators_df.drop_duplicates(inplace=True)
indicators_df.dropna(inplace=True)
indicators_df.shape

In [ ]:
#Creating a list of indicator IDs
indicators_list = indicators_df["Indicator ID"].tolist()

In [ ]:
#Removing whitespace in Indicator list
for i in range(len(indicators_list)):
    indicators_list[i] = indicators_list[i].strip()

In [ ]:
#Creating a datetime variable 13th April
data_date = (datetime.datetime(2012,4,13),datetime.datetime(2020,4,13))
data_date

In [ ]:
#Querying the World Bank Database
#world_bank_data = wbd.get_dataframe(indicators_dict,country_id,data_date)
#world_bank_data
len(indicators_list)

In [ ]:
wbd_used_cols = Series(indicators_list)
wbd_used_cols.sort_values(inplace=True)
wbd_used_cols.to_excel("wbd_used_cols.xlsx")

In [ ]:
#Checking for indicators that I can obtain data for
for i in indicators_list:
    ind_dict = {i:i}
    try:
        world_bank_data = wbd.get_dataframe(ind_dict,country_id,data_date)
    except:
        indicators_list.remove(i)
        print(i)
        
for i in indicators_list:
    ind_dict = {i:i}
    try:
        world_bank_data = wbd.get_dataframe(ind_dict,country_id,data_date)
    except:
        indicators_list.remove(i)
        print(i)

In [ ]:
len(indicators_list)

In [ ]:
indicators_dict = dict(zip(indicators_list,indicators_list))

In [ ]:
world_bank_data = wbd.get_dataframe(indicators_dict,country_id,data_date)
world_bank_data
#world_bank_data

In [ ]:
world_bank_data.columns

In [ ]:
idx = pd.IndexSlice
world_bank_data = world_bank_data.loc[idx[country_df.index, :], :]

In [ ]:
world_bank_data.head()

In [ ]:
world_bank_data.dropna(how='all',inplace=True,axis=1)

In [ ]:
world_bank_data.dropna(how='all',inplace=True)

In [ ]:
world_bank_data.head()

In [ ]:
world_bank_data.unstack().dropna(how='all',inplace=True)

In [ ]:
world_bank_data.head()

In [ ]:
world_bank_data = world_bank_data.transpose()

In [ ]:
years = ["2019","2018","2017","2016","2015","2014","2013","2012"]

In [ ]:
#Finding the year with the least nan values for every variable
total_nans = Series(np.zeros(len(years)),index=years)
new_df_cols = []
for i in world_bank_data.index:
    for j in range(0,len(years)):
        total_nans[total_nans.index[j]] += world_bank_data.loc[i, idx[:, years[j]]].isna().sum()
    col_keep = total_nans.idxmin()
    total_nans = Series(np.zeros(len(years)),index=years)
    new_df_cols.append(col_keep)

In [ ]:
world_bank_data = world_bank_data.transpose()
world_bank_data.head()

In [ ]:
world_bank_data = world_bank_data.unstack()
world_bank_data.head()

In [ ]:
#Dropping the unnecessary columns
new_df_cols = Series(new_df_cols,index=world_bank_data.stack().columns)
for i in range(len(new_df_cols)):
    drop_cols = [x for x in years if x != new_df_cols[i]]
    for col in drop_cols:
        world_bank_data.drop((new_df_cols.index[i], col), axis = 1,inplace=True)

In [ ]:
world_bank_data.head()

In [ ]:
world_bank_data = world_bank_data.droplevel(1,axis=1)

In [ ]:
#Arranging the columns in alphabetical order
world_bank_data = world_bank_data.reindex(sorted(world_bank_data.columns), axis=1)
world_bank_data.head()

In [ ]:
#filling NaN values with the mean of the columns
world_bank_data.fillna(value=world_bank_data.mean(),inplace=True)
world_bank_data.head()

In [ ]:
world_bank_data.shape

In [ ]:
world_bank_data.isna().sum().sum()

In [ ]:
#Creating an excel df of world bank variables actually used
indicators_df["Indicator ID"] = indicators_df["Indicator ID"].str.strip()
indicators_df.set_index("Indicator ID",inplace=True)

indicators_df.sort_index(inplace=True)
indicators_df.head()

indicators_df.drop_duplicates(inplace=True)
indicators_df.loc[world_bank_data.columns,:]

In [ ]:
indicators_df.head()

In [ ]:
#Reading in the "time since first case data" from worldometer.info/coronavirus
time_data = pd.read_excel("Time since first case.xlsx")
time_data.head()

In [ ]:
#remapping the country names as before
time_data.Country = time_data.Country.map(country_map_no_id).fillna(time_data.Country)
time_data.Country = time_data.Country.map(country_map_multi).fillna(time_data.Country)
time_data.Country

In [ ]:
time_data.set_index("Country",inplace=True)
world_bank_data_1 = world_bank_data.merge(time_data["Days Since First Case"],left_index=True,right_index=True)
world_bank_data_1

In [ ]:
#Reading in the data from Oxford (ourworldindata.org/coronavirus)
ox_data = pd.read_csv("owid-covid-data.csv")

ox_data.head()

In [ ]:
ox_data.set_index("date",inplace=True)

ox_data.sort_index()

In [ ]:
ox_data.columns

In [ ]:
ox_data_ovr = ox_data.loc["2020-04-13"].copy()

In [ ]:
ox_data_ovr.head()

In [ ]:
ox_data_ovr.reset_index(inplace=True)

In [ ]:
iso_code_map = dict(zip(country_df["WBD ID"],country_df["WBD Name"]))
ox_data_ovr.iso_code = ox_data_ovr.iso_code.map(iso_code_map)

In [ ]:
ox_data_ovr.reset_index(inplace=True)

In [ ]:
ox_data_ovr.set_index("iso_code",inplace=True)

In [ ]:
ox_data_ovr.sort_index(inplace=True)
ox_data_ovr.head()

In [ ]:
ox_data_ovr.columns

In [ ]:
#dropping unnecessary columns from Oxford database
ox_data_ovr.drop(['date', 'location', 'new_cases',
       'new_deaths', 'total_cases_per_million', 'new_cases_per_million',
       'total_deaths_per_million', 'new_deaths_per_million','tests_units','total_tests_per_thousand','new_tests_per_thousand',\
                 'female_smokers']\
                 ,axis=1,inplace=True)

In [ ]:
ox_data_ovr.columns

In [ ]:
ox_data_ovr.head(15)

In [ ]:
ox_data_ovr[["total_deaths"]].fillna(value=0,inplace=True)
ox_data_ovr.total_cases.isna().sum()

In [ ]:
ox_data_ovr.fillna(value=ox_data_ovr.mean(),inplace=True)

In [ ]:
ox_data_ovr.head(15)

In [ ]:
ox_data_ovr.shape

In [ ]:
data_ovr = ox_data_ovr.merge(world_bank_data_1,left_index=True,right_index=True)
data_ovr.head()

In [ ]:
data_ovr.shape

In [ ]:
data_ovr.dropna(inplace=True,how='all')

In [ ]:
data_ovr.shape

In [ ]:
total_cases_pc = data_ovr["total_cases"]/data_ovr["population"]
total_deaths_pc = data_ovr["total_deaths"]/data_ovr["population"]

In [ ]:
#Calculating per capita values for below variables
data_ovr[["total_tests","EN.ATM.GHGT.KT.CE","IS.AIR.PSGR","SH.DTH.0514",]]=\
data_ovr[["total_tests","EN.ATM.GHGT.KT.CE","IS.AIR.PSGR","SH.DTH.0514",]]\
.div(data_ovr["population"],axis=0)

In [ ]:
data_ovr.drop(['index','population',"new_tests","aged_65_older","cvd_death_rate"],axis=1,inplace=True)

In [ ]:
data_ovr.head()

In [ ]:
data_ovr.total_cases.isna().sum()

In [ ]:
data_ovr.shape

In [ ]:
#used_columns = data_ovr.columns
#used_columns = Series(used_columns.values)
#used_columns.sort_values(inplace=True)
#used_columns.to_excel("Variable Data.xlsx")

In [ ]:
#Creating the explanatory variables dataframes (X1 for cases per capita, X2 for deaths per capita)
X1 = data_ovr.drop(["total_cases",'total_deaths'],axis=1).copy()
X1 = sm.add_constant(X1)
X2 = data_ovr.drop(["total_tests",'total_deaths'],axis=1).copy()
X2 = sm.add_constant(X2)

In [ ]:
#Performing overall "raw" regressions
mod_raw_cases = sm.OLS(total_cases_pc,X1).fit(cov_type='HAC',cov_kwds={'maxlags':12,'use_correction':True})
mod_raw_deaths = sm.OLS(total_deaths_pc,X2).fit(cov_type='HAC',cov_kwds={'maxlags':12,'use_correction':True})
mod_raw_cases.summary()

In [ ]:
sum_raw_cases = mod_raw_cases.summary2()
sum_raw_cases_table = sum_raw_cases.tables[1]
#sum_raw_cases_table

In [ ]:
sum_raw_deaths = mod_raw_deaths.summary2()
sum_raw_deaths_table = sum_raw_deaths.tables[1]
#sum_raw_deaths_table

In [ ]:
raw_summary = summary_col([mod_raw_cases,mod_raw_deaths],float_format='%0.5f',info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),\
                            'R2':lambda x: "{:.4f}".format(x.rsquared),'AIC':lambda x: "{:.4f}".format(x.aic),\
                                                             'BIC':lambda x: "{:.4f}".format(x.bic),\
                            'Condition Number':lambda x: "{0:d}".format(int(x.condition_number))},stars=True).tables[0]
raw_summary

In [ ]:
#Sorting the two summary tables
sum_raw_cases_table.sort_index(inplace=True)
sum_raw_deaths_table.sort_index(inplace=True)

In [ ]:
#Calculating the variance inflation factor to check for multicollinearity and remove variables
vif_cases = DataFrame()
vif_cases["VIF Factor"] = [variance_inflation_factor(X1.values, i) for i in range(X1.shape[1])]
vif_cases["features"] = X1.columns

vif_deaths = DataFrame()
vif_deaths["VIF Factor"] = [variance_inflation_factor(X2.values, i) for i in range(X2.shape[1])]
vif_deaths["features"] = X2.columns

In [ ]:
vif_cases.sort_values("VIF Factor",ascending=False,inplace=True)
vif_deaths.sort_values("VIF Factor",ascending=False,inplace=True)

In [ ]:
vif_cases = vif_cases[vif_cases.features != "const"]
vif_deaths = vif_deaths[vif_deaths.features != "const"]

In [ ]:
vif_deaths[vif_deaths["VIF Factor"]>10].features

In [ ]:
X1.drop(columns=vif_cases[vif_cases["VIF Factor"]>10].features,inplace=True)
X2.drop(columns=vif_deaths[vif_deaths["VIF Factor"]>10].features,inplace=True)

vif_cases = DataFrame()
vif_cases["VIF Factor"] = [variance_inflation_factor(X1.values, i) for i in range(X1.shape[1])]
vif_cases["features"] = X1.columns

vif_deaths = DataFrame()
vif_deaths["VIF Factor"] = [variance_inflation_factor(X2.values, i) for i in range(X2.shape[1])]
vif_deaths["features"] = X2.columns

In [ ]:
vif_cases.sort_values("VIF Factor",ascending=False,inplace=True)
vif_deaths.sort_values("VIF Factor",ascending=False,inplace=True)

In [ ]:
#Stepwise Forward Regressions - feature reduction
#Each of these functions output feature names based on the AIC and BIC criterion
def AIC_step(X,y):
    indep_vars = ['const']
    aics = [0]
    X_1 = X.copy()
    X_2 = X.copy()
    while aics[0] <= 0:
        reg_intercept_only = sm.OLS(y,X_1.loc[:,indep_vars]).fit()
        base_aic = reg_intercept_only.aic
        aics = Series(0.0,index=X_2.columns[1:])
        for i in X_2.columns[1:]:
            cols = indep_vars + [i]
            reg = sm.OLS(y,X_1.loc[:,cols]).fit()
            aics[i] += reg.aic
        aics -= base_aic
        aics.sort_values(inplace=True)
        if aics[0] < 0:
            indep_vars.append(aics.index[0])
        X_2.drop(aics.index[0],axis=1,inplace=True)    
    return indep_vars

def BIC_step(X,y):
    indep_vars = ['const']
    bics = [0]
    X_1 = X.copy()
    X_2 = X.copy()
    while bics[0] <= 0:
        reg_intercept_only = sm.OLS(y,X_1.loc[:,indep_vars]).fit()
        base_bic = reg_intercept_only.bic
        bics = Series(0.0,index=X_2.columns[1:])
        for i in X_2.columns[1:]:
            cols = indep_vars + [i]
            reg = sm.OLS(y,X_1.loc[:,cols]).fit()
            bics[i] += reg.bic
        bics -= base_bic
        bics.sort_values(inplace=True)
        if bics[0] < 0:
            indep_vars.append(bics.index[0])
        X_2.drop(bics.index[0],axis=1,inplace=True)    
    return indep_vars

In [ ]:
#Performing the regressions on the reduced feature datasets
X1_aic = X1.loc[:,AIC_step(X1,total_cases_pc)].copy()
X2_aic = X2.loc[:,AIC_step(X2,total_deaths_pc)].copy()

X1_bic = X1.loc[:,BIC_step(X1,total_cases_pc)].copy()
X2_bic = X2.loc[:,BIC_step(X2,total_deaths_pc)].copy()

In [ ]:
mod_cases_aic = sm.OLS(total_cases_pc,X1_aic).fit(cov_type='HAC',cov_kwds={'maxlags':12,'use_correction':True})
mod_cases_bic = sm.OLS(total_cases_pc,X1_bic).fit(cov_type='HAC',cov_kwds={'maxlags':12,'use_correction':True})

mod_deaths_aic = sm.OLS(total_deaths_pc,X2_aic).fit(cov_type='HAC',cov_kwds={'maxlags':12,'use_correction':True})
mod_deaths_bic = sm.OLS(total_deaths_pc,X2_bic).fit(cov_type='HAC',cov_kwds={'maxlags':12,'use_correction':True})

step_summary = summary_col([mod_cases_aic,mod_cases_bic,mod_deaths_aic,mod_deaths_bic],float_format='%0.5f',info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),\
                            'R2':lambda x: "{:.4f}".format(x.rsquared),'AIC':lambda x: "{:.4f}".format(x.aic),\
                                                             'BIC':lambda x: "{:.4f}".format(x.bic)},stars=True).tables[0]
step_summary

In [ ]:
#Performing Lasso Regressions

In [ ]:
#Calculating the optimal alpha parameter and coefficients for the lasso regressions using the AIC criterion
lasso_aic = lasso_ic(criterion='aic',normalize=True)
aic_cases_alpha = lasso_aic.fit(X1,total_cases_pc).alpha_
aic_cases_coef = lasso_aic.fit(X1,total_cases_pc).coef_.nonzero()[0]
X1_lasso_aic = X1.iloc[:,aic_cases_coef].copy()
X1_lasso_aic = sm.add_constant(X1_lasso_aic)

aic_deaths_alpha = lasso_aic.fit(X2,total_deaths_pc).alpha_
aic_deaths_coef = lasso_aic.fit(X2,total_deaths_pc).coef_.nonzero()[0]
X2_lasso_aic = X2.iloc[:,aic_deaths_coef].copy()
X2_lasso_aic = sm.add_constant(X2_lasso_aic)

In [ ]:
#Calculating the optimal alpha parameter and coefficients for the lasso regressions using the BIC criterion
lasso_bic = lasso_ic(criterion='bic',normalize=True)
bic_cases_alpha = lasso_bic.fit(X1,total_cases_pc).alpha_
bic_cases_coef = lasso_bic.fit(X1,total_cases_pc).coef_.nonzero()[0]
X1_lasso_bic = X1.iloc[:,bic_cases_coef].copy()
X1_lasso_bic = sm.add_constant(X1_lasso_bic)

bic_deaths_alpha = lasso_bic.fit(X2,total_deaths_pc).alpha_
bic_deaths_coef = lasso_bic.fit(X2,total_deaths_pc).coef_.nonzero()[0]
X2_lasso_bic = X2.iloc[:,bic_deaths_coef].copy()
X2_lasso_bic = sm.add_constant(X2_lasso_bic)

In [ ]:
#Calculating the optimal alpha parameter and coefficients for the Lasso regressions using Cross Validation
lasso_cv = LassoCV(random_state=23,normalize=True)
cv_cases_alpha = lasso_cv.fit(X1,total_cases_pc).alpha_
cv_cases_coef = lasso_cv.fit(X1,total_cases_pc).coef_.nonzero()[0]
X1_lasso_cv = X1.iloc[:,cv_cases_coef].copy()
X1_lasso_cv = sm.add_constant(X1_lasso_cv)

cv_deaths_aplha = lasso_cv.fit(X2,total_deaths_pc).alpha_
cv_deaths_coef = lasso_cv.fit(X2,total_deaths_pc).coef_.nonzero()[0]
X2_lasso_cv = X2.iloc[:,cv_deaths_coef].copy()
X2_lasso_cv = sm.add_constant(X2_lasso_cv)

In [ ]:
#Lasso Regressions
lasso_cases_aic = sm.OLS(total_cases_pc,X1_lasso_aic).fit_regularized(alpha=aic_cases_alpha,L1_wt=1,refit=True)
lasso_cases_bic = sm.OLS(total_cases_pc,X1_lasso_bic).fit_regularized(alpha=bic_cases_alpha,L1_wt=1,refit=True)
lasso_cases_cv = sm.OLS(total_cases_pc,X1_lasso_cv).fit_regularized(alpha=cv_cases_alpha,L1_wt=1,refit=True)

lasso_deaths_aic = sm.OLS(total_deaths_pc,X2_lasso_aic).fit_regularized(alpha=aic_deaths_alpha,L1_wt=1,refit=True)
lasso_deaths_bic = sm.OLS(total_deaths_pc,X2_lasso_bic).fit_regularized(alpha=bic_deaths_alpha,L1_wt=1,refit=True)
lasso_deaths_cv = sm.OLS(total_deaths_pc,X2_lasso_cv).fit_regularized(alpha=cv_deaths_aplha,L1_wt=1,refit=True)

In [ ]:
lasso_summary = summary_col([lasso_cases_aic,lasso_cases_bic,lasso_cases_cv,lasso_deaths_aic,lasso_deaths_bic,lasso_deaths_cv]\
                            ,float_format='%0.5f',info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),\
                            'R2':lambda x: "{:.4f}".format(x.rsquared),'AIC':lambda x: "{:.4f}".format(x.aic),\
                                                             'BIC':lambda x: "{:.4f}".format(x.bic)},stars=True).tables[0]
lasso_summary

In [ ]:
#Exporting results from overall,AIC/BIC and Lasso Regressions
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('covid_regressions_results.xlsx')

raw_summary.to_excel(writer, sheet_name="raw_regressions")
step_summary.to_excel(writer, sheet_name="stepwise_regressions")
lasso_summary.to_excel(writer, sheet_name="lasso_regressions")

writer.save()